# Import Modules

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm.notebook import tqdm
from collections import namedtuple
from collections import defaultdict
from pprint import pprint

# Configure display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
%config InlineBackend.figure_format = 'retina'

# Add project directory to sys.path
# pdir = os.path.abspath(os.path.join(os.getcwd(), '.'))
# sys.path.insert(1, pdir)
# from myutils import *

# Other module imports
import ast, math, swifter, csv, json, itertools as it, portion as P

# Set plot style
# plt.style.use('ggplot')

# Function Definitions

In [187]:
# ************************************************* unify data format *************************************************

def set_data(df, mode='pcap', tz=0):
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    
    if mode == 'pcap':
        common_column_names = [
            'seq', 'rpkg', 'frame_id', 'Timestamp', 'Timestamp_epoth', 'lost', 'excl', 'latency',
            'xmit_time', 'xmit_time_epoch', 'arr_time', 'arr_time_epoch',
        ]
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        date_columns = ['Timestamp', 'xmit_time', 'arr_time']
        df[date_columns] = df[date_columns].apply(pd.to_datetime)
        df[['seq', 'rpkg', 'frame_id']] = df[['seq', 'rpkg', 'frame_id']].astype('Int32')
        df[['Timestamp_epoch', 'xmit_time_epoch', 'arr_time_epoch', 'latency']] = \
            df[['Timestamp_epoch', 'xmit_time_epoch', 'arr_time_epoch', 'latency']].astype('float32')
        df[['lost', 'excl']] = df[['lost', 'excl']].astype('boolean')

    if mode in ['lte', 'nr']:
        common_column_names = [
            'Timestamp', 'type_id', 'PCI', 'RSRP', 'RSRQ', 'serv_cel_index', 'EARFCN', 'NR_ARFCN', 
            'num_cels', 'num_neigh_cels', 'serv_cel_pos', 'PCI0', 'RSRP0', 'RSRQ0',
        ]
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        if mode == 'lte':
            columns_mapping = {
                'RSRP(dBm)': 'RSRP',
                'RSRQ(dB)': 'RSRQ',
                'Serving Cell Index': 'serv_cel_index',
                'Number of Neighbor Cells': 'num_neigh_cels',
                'Number of Detected Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df['serv_cel_index'] = np.where(df['serv_cel_index'] == '(MI)Unknown', '3_SCell', df['serv_cel_index'])
            df['num_cels'] = df['num_neigh_cels'] + 1
            df['type_id'] = 'LTE_PHY'

        if mode == 'nr':
            columns_mapping = {
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df[['PCI', 'RSRP', 'RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df['serv_cel_index'] = np.where(df['serv_cel_pos'] == 255, df['serv_cel_index'], 'PSCell')
            df['num_neigh_cels'] = np.where(df['serv_cel_pos'] == 255, df['num_cels'], df['num_cels'] - 1)
            df['type_id'] = '5G_NR_ML1'
        
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df[['type_id', 'serv_cel_index']] = df[['type_id', 'serv_cel_index']].astype('category')
        df[['EARFCN', 'NR_ARFCN']] = df[['EARFCN', 'NR_ARFCN']].astype('Int32')
        df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']] = df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']].astype('UInt8')

        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP', 'RSRQ')):
                df[tag] = df[tag].astype('float32')

    return df

# ************************************************* Sheng-Ru Latest Version (09-25) add try except (10-27) *************************************************

def parse_mi_ho(df, tz=8):

    # df = pd.read_csv(f)
    df["Timestamp"] = df["Timestamp"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=tz))
    nr_pci = 'O'
    scells = []

    def NR_OTA(idx):

        if df["type_id"].iloc[idx] == "5G_NR_RRC_OTA_Packet": return True
        else: return False
    
    def LTE_SERV_INFO(idx):

        if df["type_id"].iloc[idx] == "LTE_RRC_Serv_Cell_Info": return True
        else: return False
    

    def find_1st_after(start_idx, target, look_after=1):
        for j in range(start_idx, len(df)):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t_ - t).total_seconds() > look_after:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before(start_idx, target, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before_with_special_value(start_idx, target, target_value, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] in [target_value] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_in_D_exact(targets):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.start).total_seconds(), target))

        if len(l) != 0:
            for x in l:
                if (x[0]== 0):
                    return x[1]
        
        return None
    
    def find_in_D_first_before(targets, look_before=1):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                try:
                    l.append(((t - ho.end).total_seconds(), target, ho))
                except:
                    pass

        if len(l) != 0:
            closest = min(filter(lambda x: x[0] > 0, l), key=lambda x: x[0])
            if 0 <= closest[0] < look_before:
                return closest[1], closest[2]
        
        return None, None
    
    HO = namedtuple('HO',['start', 'end', 'others', 'trans'], defaults=[None,None,'',''])
    
    D = {
        'Conn_Rel':[], 
        'Conn_Req':[], # Setup
        'LTE_HO': [], # LTE -> newLTE
        'MN_HO': [], # LTE + NR -> newLTE + NR
        'MN_HO_to_eNB': [], # LTE + NR -> newLTE
        'SN_setup': [], # LTE -> LTE + NR => NR setup
        'SN_Rel': [], # LTE + NR -> LTE
        'SN_HO': [], # LTE + NR -> LTE + newNR  
        'RLF_II': [],
        'RLF_III': [],
        'SCG_RLF': [],
        'Add_SCell': [],
        }

    for i in range(len(df)):

        # Pass NR RRC packet. In NSA mode, LTE RRC packet include NR packet message.
        if NR_OTA(i) or LTE_SERV_INFO(i):
            continue

        others = ''
        t = df["Timestamp"].iloc[i]

        if df["rrcConnectionRelease"].iloc[i] == 1:
            D['Conn_Rel'].append(HO(start=t))
            nr_pci = 'O'

        if df["rrcConnectionRequest"].iloc[i] == 1:
            
            # Define end of rrcConnectionRequest to be rrcConnectionReconfigurationComplete or securityModeComplete.
            a = find_1st_after(i, 'rrcConnectionReconfigurationComplete',look_after=2)[0]
            b = find_1st_after(i, 'securityModeComplete',look_after=2)[0]
            if a is None and b is None: end = None
            elif a is None and b is not None: end = b
            elif a is not None and b is None: end = a 
            else: end = a if a > b else b
            
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            trans = f'? -> ({serv_cell}, {serv_freq})'
            D['Conn_Req'].append(HO(start=t,end=end,trans=trans))
            nr_pci = 'O'
        
        if df["lte-rrc.t304"].iloc[i] == 1:
            
            end, _ = find_1st_after(i, 'rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = df["PCI"].iloc[i], int(df['lte_targetPhysCellId'].iloc[i])
            serv_freq, target_freq = int(df["Freq"].iloc[i]), int(df['dl-CarrierFreq'].iloc[i])

            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others += f' Set up {n} SCell.'
            else:
                scells = []
            
            if serv_freq != target_freq:
                a,b = find_1st_before(i, "rrcConnectionReestablishmentRequest", 1)
                others += " Inter frequency HO."
                if a is not None:
                    others += " Near after RLF."
                
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 2)
                    
                    if a is not None:

                        ho_type, ho = find_in_D_first_before(['RLF_II', 'RLF_III'], 2)
                        try:
                            others += f' Near after RLF of trans: {ho.trans}.'
                        except:
                            others += f' Near after RLF.'

                    else:
                        
                        ho_type, _ = find_in_D_first_before(['MN_HO_to_eNB', 'SN_Rel'], 2)
                        if ho_type is not None:
                            others += f' Near after {ho_type}.'

                    ori_serv = nr_pci
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) | {ori_serv} -> {nr_pci}'
                    D['SN_setup'].append(HO(start=t, end=end, others=others, trans=trans))

                else:
                    
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                    D['MN_HO'].append(HO(start=t, end=end, others=others, trans=trans))

            else:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, b = find_1st_before(i, "scgFailureInformationNR-r15")
                    if a is not None:
                        others += " Caused by scg-failure."
                    
                    orig_serv = nr_pci
                    nr_pci = 'O'
                    trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                    D['SN_Rel'].append(HO(start=t, end=end, others=others, trans=trans))
                    
                else:

                    a, _ = find_1st_before(i,"rrcConnectionSetup",3)
                    if a is not None:
                        others += ' Near After connection setup.'
                    if nr_pci == 'O':
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                        D['LTE_HO'].append(HO(start=t, end=end, others=others, trans=trans))
                    else:
                        orig_serv = nr_pci
                        nr_pci = 'O'
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {orig_serv} -> {nr_pci}'
                        D['MN_HO_to_eNB'].append(HO(start=t, end=end, others=others, trans=trans))


        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
        
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            orig_serv = nr_pci
            nr_pci = int(df['nr_physCellId'].iloc[i])
            trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
            D['SN_HO'].append(HO(start=t,end=end,trans=trans))


        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:

            end1, _ = find_1st_after(i, 'rrcConnectionReestablishmentComplete', look_after=1)
            b, _ = find_1st_after(i, 'rrcConnectionReestablishmentReject', look_after=1)
            end2, _ = find_1st_after(i, 'securityModeComplete',look_after=3)

            others += ' ' + df["reestablishmentCause"].iloc[i] + '.'
            scells = []

            c, _ = find_1st_before(i, 'scgFailureInformationNR-r15', 1)
            if c != None:
                others  += ' caused by scgfailure.'
                
            serv_cell, rlf_cell = df["PCI"].iloc[i], int(df['physCellId.3'].iloc[i])
            serv_freq = int(df['Freq'].iloc[i])
            
            # Type II & Type III
            if end1 is not None: 

                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                try:
                    rlf_freq = int(df['Freq'].iloc[idx])
                except:
                    rlf_freq = 0
                trans = f'({rlf_cell}, {rlf_freq}) -> ({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_II'].append(HO(start=t,end=end1,others=others,trans=trans))

            elif b is not None and end2 is not None:
                
                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                try:
                    rlf_freq = int(df['Freq'].iloc[idx])
                except:
                    rlf_freq = 0

                _, idx = find_1st_after(i, "rrcConnectionRequest", 2)
                recon_cell, recon_freq = df['PCI'].iloc[idx], int(float(df['Freq'].iloc[idx]))
                
                trans = f'({rlf_cell}, {rlf_freq}) -> ({recon_cell}, {recon_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_III'].append(HO(start=t,end=end2,others=others,trans=trans))
                
            else:
                others+=' No end.'
                D['RLF_II'].append(HO(start=t,others=others))
                print('No end for RLF')

        if df["scgFailureInformationNR-r15"].iloc[i] == 1:

            others += ' ' + df["failureType-r15"].iloc[i] + '.'
            a, idx1 = find_1st_after(i, "rrcConnectionReestablishmentRequest", look_after=1)
            b, idx2 = find_1st_after(i, "lte-rrc.t304", look_after=10)

            if a is not None:

                end1, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentComplete', look_after=1)
                b, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentReject', look_after=1)
                end2 = find_1st_after(idx1, 'securityModeComplete',look_after=3)[0]

                others += ' Result in rrcReestablishment.'
                    
                # Type II & Type III Result
                if end1 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end1,others=others))
                elif b is not None and end2 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end2,others=others))
                else:
                    others += ' No end.'
                    D['SCG_RLF'].append(HO(start=t,others=others))
                    print('No end for scg failure result in rrcReestablishment.')

            elif b is not None:

                end, _ = find_1st_after(idx2, 'rrcConnectionReconfigurationComplete')
                serv_cell, target_cell = df["PCI"].iloc[idx2], df['lte_targetPhysCellId'].iloc[idx2]
                serv_freq, target_freq = int(df["Freq"].iloc[idx2]), df['dl-CarrierFreq'].iloc[idx2]
                others += ' Result in gNB release.'
                # We do not change nr_pci here. Instead, we will change it at gNB_Rel event.
                trans = f'({serv_cell}, {serv_freq}) | {nr_pci} -> O'
                
                if serv_cell == target_cell and serv_freq == target_freq:
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))
                else:
                    others += ' Weird gNB release.'
                    print('Weird for scg failure result in gNb Release.')
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))                  

            else:

                print('No end for scg failure.')
                others += ' No end.'
                D['SCG_RLF'].append(HO(start=t,others=others))
        
        if df['SCellToAddMod-r10'].iloc[i] == 1 and df['physCellId-r10'].iloc[i] != 'nr or cqi report':

            others = ''
            pcis = str(df["physCellId-r10"].iloc[i]).split('@')
            freqs = str(df["dl-CarrierFreq-r10"].iloc[i]).split('@')
            orig_scells = scells
            scells = [(int(float(pci)), int(float(freq))) for pci, freq in zip(pcis, freqs)]

            others += f' Set up {len(scells)} SCell.'
            trans = f'{orig_scells} -> {scells}'

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
            
            a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 3)
            if a is not None:
                others += ' Near after RLF.'

            a = find_in_D_exact(['LTE_HO', 'MN_HO', 'MN_HO_to_eNB', 'SN_setup', 'SN_Rel'])
            if a is not None:
                others += f' With {a}.'

            D['Add_SCell'].append(HO(start=t,end=end,others=others, trans=trans))
    
    return D

# ************************************************* parse into readable dataframe *************************************************

def mi_parse_handover(df, tz=8, radical=True):
    
    def parse_trans(item):
        
        chunk = item.split(' | ')
        
        if len(chunk) == 1:
            s_src = np.nan
            s_tgt = np.nan
            if chunk[0] == '':
                m_src = np.nan
                m_tgt = np.nan
            elif chunk[0][0] == '?':
                m_src = np.nan
                m_tgt = chunk[0].split(' -> ')[1]
            else:
                m_src = chunk[0].split(' -> ')[0]
                m_tgt = chunk[0].split(' -> ')[1]
        else:
            if chunk[1] == 'O':
                s_src = np.nan
                s_tgt = np.nan
            else:
                chunk1 = chunk[1].split(' -> ')
                if len(chunk1) == 1:
                    s_src = chunk1[0]
                    s_tgt = np.nan
                else:
                    s_src = chunk1[0] if chunk1[0] != 'O' else np.nan
                    s_tgt = chunk1[1] if chunk1[1] != 'O' else np.nan
                
            chunk1 = chunk[0].split(' -> ')
            if len(chunk1) == 1:
                m_src = chunk1[0]
                m_tgt = np.nan
            else:
                m_src = chunk1[0]
                m_tgt = chunk1[1]
                
        return m_src, m_tgt, s_src, s_tgt
    
    key_mapping = {
        'Conn_Rel': 'CXNR',
        'Conn_Req': 'CXNS',
        'LTE_HO': 'LTEH',
        'MN_HO': 'MCGH',
        'MN_HO_to_eNB': 'SCGR-II',
        'SN_setup': 'SCGA',
        'SN_Rel': 'SCGR-I',
        'SN_HO': 'SCGM',
        'RLF_II': 'MCGF',
        'RLF_III': 'NASR',
        'SCG_RLF': 'SCGF',
        'Add_SCell': 'SCLA'
    }
    
    D = parse_mi_ho(df, tz)
    
    # rename as acronym
    new_D = {key_mapping.get(key, key): value for key, value in D.items()}
    
    if df.empty:
        selected_cols = ['PCI', 'Cell Identity', 'eNB_ID', 'next_eNB_ID', 'Band ID', 'next_BID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
        table = pd.DataFrame(columns=['type', 'start', 'end', 'others', 'm_src', 'm_tgt', 's_src', 's_tgt', 'category', 'inter-eNB', 'inter-Freq', 'band_cng', 'inter-RAT', '4G_5G', 'cause', 'near_before_RLF', *selected_cols])
        print('Empty RRC File!!!')
        return table, new_D
    
    table = pd.DataFrame()
    for key, lst in new_D.items():
        table1 = pd.DataFrame(lst, index=[key]*len(lst)).reset_index(names='type')
        table = pd.concat([table, table1], ignore_index=True)

    # add Cell Identity & eNB ID
    sc_info = df[df['type_id'] == 'LTE_RRC_Serv_Cell_Info'][['Timestamp', 'type_id', 'PCI', 'Cell Identity', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']].reset_index(drop=True).rename(columns={'Timestamp': 'start', 'type_id': 'type'})
    sc_info['eNB_ID'] = sc_info['Cell Identity'] // 256
    # sc_info['Cell_ID'] = sc_info['Cell Identity'] % 256
    sc_info = sc_info[['start', 'type', 'PCI', 'Cell Identity', 'eNB_ID', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']]

    table = pd.concat([table, sc_info], ignore_index=True).sort_values(by='start').reset_index(drop=True)

    is_not_start = True
    selected_cols = ['PCI', 'Cell Identity', 'eNB_ID', 'Band ID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
    for i, row in table.iterrows():
        if row['type'] == 'LTE_RRC_Serv_Cell_Info':
            is_not_start = False
            info_to_fill = row[selected_cols].to_list()
            continue
        if is_not_start:
            continue
        table.loc[i, selected_cols] = info_to_fill

    table = table[table['type'] != 'LTE_RRC_Serv_Cell_Info'].reset_index(drop=True)
    
    # parse source & target cells
    for i, row in table.iterrows():
        table.loc[i, ['m_src', 'm_tgt', 's_src', 's_tgt']] = parse_trans(row['trans'])
    
    # distinguish intra/inter-eNB HO
    table1 = table[np.in1d(table['type'], ['SCLA', 'SCGA', 'SCGR-I', 'SCGF'])]
    table = table[~np.in1d(table['type'], ['SCLA', 'SCGA', 'SCGR-I', 'SCGF'])].reset_index(drop=True)
    
    table['next_eNB'] = table['eNB_ID'].shift(-1)
    for i, row in table.iloc[:-1].iterrows():
        if row['eNB_ID'] != row['next_eNB'] and row['type'] not in ['CXNS', 'CXNR']:
            if row['others'] == '':
                table.at[i, 'others'] = 'Inter eNB HO.'
            else:
                table.at[i, 'others'] += ' Inter eNB HO.'
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # label SCG Addition near after SCG Failure
    table1 = table[~np.in1d(table['type'], ['SCGA', 'SCGR-I', 'SCGR-II'])]
    table = table[np.in1d(table['type'], ['SCGA', 'SCGR-I', 'SCGR-II'])].reset_index(drop=True)
    
    table['prev_cmt'] = table['others'].shift(1)
    for i, row in table.iloc[1:].iterrows():
        if row['type'] == 'SCGA':
            if 'Near after SN_Rel' in row['others'] and 'Caused by scg-failure' in row['prev_cmt']:
                table.at[i, 'others'] += ' Caused by scg-failure.'
    
    # combine closed SCG Addition & Release pair (which are not caused by scg-failure or RLF) into SCG Change
    table['next_end'] = table['end'].shift(-1)
    table['next_cmt'] = table['others'].shift(-1)
    table['next_s_tgt'] = table['s_tgt'].shift(-1)
    indices_to_remove = []
    for i, row in table.iloc[:-1].iterrows():
        if row['type'] == 'SCGR-I' and 'Near after SN_Rel' in row['next_cmt'] and row['s_src'] != row['next_s_tgt']:
            table.at[i, 'end'] = row['next_end']
            table.at[i, 's_tgt'] = row['next_s_tgt']
            table.at[i, 'type'] = 'SCGC-I'
            indices_to_remove.append(i+1)
        if row['type'] == 'SCGR-II' and 'Near after MN_HO_to_eNB' in row['next_cmt'] and row['s_src'] != row['next_s_tgt']:
            table.at[i, 'end'] = row['next_end']
            table.at[i, 's_tgt'] = row['next_s_tgt']
            table.at[i, 'type'] = 'SCGC-II'
            indices_to_remove.append(i+1)
    table = table.drop(indices_to_remove)
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # re-classify eNB HO & MeNB HO
    table.loc[np.in1d(table['type'], ['LTEH']) & table['others'].str.contains('Inter eNB HO'), 'type'] = 'ENBH'
    table.loc[np.in1d(table['type'], ['MCGH']) & table['others'].str.contains('Inter eNB HO'), 'type'] = 'MNBH'
    
    # add the next eNB ID when meeting inter-eNB HO
    table1 = table[~table['others'].str.contains('Inter eNB HO')]
    table = table[table['others'].str.contains('Inter eNB HO')].reset_index(drop=True)
    
    table['next_eNB_ID'] = table['eNB_ID'].shift(-1)
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # detect band change and add the next Band ID when meeting inter-Freq HO
    band_mapping = {}
    for i, row in table[~table.duplicated(subset=['DL frequency'])].dropna(subset=['DL frequency']).iterrows():
        band_mapping[int(row['DL frequency'])] = row['Band ID']
    
    print(band_mapping)
    
    table1 = table[~table['others'].str.contains('Inter frequency HO')]
    table = table[table['others'].str.contains('Inter frequency HO')].reset_index(drop=True)
    
    table['next_BID'] = table['Band ID'].shift(-1)
    try:
        table.at[len(table)-1, 'next_BID'] = band_mapping[ast.literal_eval(table.iloc[-1]['m_tgt'])[1]]
    except:
        pass
    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # check whether RLF is near after an HO event
    table1 = table[np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])]
    table = table[~np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])].reset_index(drop=True)
    
    table['next_start'] = table['start'].shift(-1)
    table['next_type'] = table['type'].shift(-1)
    table['near_before_RLF'] = False
    for i, row in table.iloc[:-1].iterrows():
        # check whether an HO event is near before an RLF (in 3 seconds)
        if row['type'] in ['LTEH', 'ENBH', 'MCGH', 'MNBH', 'SCGM', 'SCGA', 'SCGR-I', 'SCGR-II', 'SCGC-I', 'SCGC-II'] and \
            row['next_type'] in ['NASR', 'MCGF', 'SCGF']:
                if row['end'] > row['next_start'] - pd.Timedelta(seconds=1):
                    table.at[i, 'near_before_RLF'] = True
                    next_type = row['next_type']
                    if row['others'] == '':
                        table.at[i, 'others'] = f'Near before {next_type} 1 sec.'
                    else:
                        table.at[i, 'others'] += f' Near before {next_type} 1 sec.'
                        
                elif row['end'] > row['next_start'] - pd.Timedelta(seconds=2):
                    table.at[i, 'near_before_RLF'] = True
                    next_type = row['next_type']
                    if row['others'] == '':
                        table.at[i, 'others'] = f'Near before {next_type} 2 sec.'
                    else:
                        table.at[i, 'others'] += f' Near before {next_type} 2 sec.'
                        
                elif row['end'] > row['next_start'] - pd.Timedelta(seconds=3):
                    table.at[i, 'near_before_RLF'] = True
                    next_type = row['next_type']
                    if row['others'] == '':
                        table.at[i, 'others'] = f'Near before {next_type} 3 sec.'
                    else:
                        table.at[i, 'others'] += f' Near before {next_type} 3 sec.'
                    
    table = pd.concat([table, table1], ignore_index=True).sort_values(by='start').reset_index(drop=True)
    
    # add category
    table['category'] = 'Others'
    table.loc[np.in1d(table['type'], ['LTEH', 'ENBH', 'MCGH', 'MNBH', 'SCGM', 'SCGA', 'SCGR-I', 'SCGC-I', 'SCGR-II', 'SCGC-II']), 'category'] = 'HO'
    table.loc[np.in1d(table['type'], ['MCGF', 'NASR', 'SCGF']), 'category'] = 'RLF'

    # add failure cause
    failure_cause = [
        'reconfigurationFailure (0)', 'handoverFailure (1)', 'otherFailure (2)',
        't310-Expiry (0)', 'randomAccessProblem (1)', 'rlc-MaxNumRetx (2)', 'synchReconfigFailureSCG (3)', 'scg-ReconfigFailure (4)', 'srb3-IntegrityFailure (5)', 'other-r16 (6)'
    ]
    
    for tag in failure_cause:
        table.loc[table['others'].str.contains(tag, regex=False), 'cause'] = tag
        table['others'] = table['others'].str.replace(f" {tag}.", "", regex=False)
        table['others'] = table['others'].str.replace(f"{tag}.", "", regex=False)
    
    # add Access Technology type
    table['4G_5G'] = '4G'
    table.loc[np.in1d(table['type'], ['SCGM', 'SCGA', 'SCGR-I', 'SCGC-I', 'SCGF']), '4G_5G'] = '5G'
    table.loc[np.in1d(table['type'], ['SCGR-II', 'SCGC-II']), '4G_5G'] = '4G_5G'
    
    # add more boolean columns
    table['inter-eNB'] = False
    table.loc[table['others'].str.contains('Inter eNB HO'), 'inter-eNB'] = True
    table['others'] = table['others'].str.replace(" Inter eNB HO.", "")
    table['others'] = table['others'].str.replace("Inter eNB HO.", "")
    
    table['inter-Freq'] = False
    table.loc[table['others'].str.contains('Inter frequency HO'), 'inter-Freq'] = True
    table['others'] = table['others'].str.replace(" Inter frequency HO.", "")
    table['others'] = table['others'].str.replace("Inter frequency HO.", "")
    
    table['band_cng'] = False
    table.loc[table['inter-Freq'] & (table['Band ID'] != table['next_BID']), 'band_cng'] = True
    
    table['inter-RAT'] = False
    table.loc[np.in1d(table['type'], ['SCGA', 'SCGR-I', 'SCGC-I', 'SCGR-II', 'SCGC-II']), 'inter-RAT'] = True
    
    table['inter-gNB'] = False
    table.loc[np.in1d(table['type'], ['SCGC-I', 'SCGC-II']), 'inter-gNB'] = True
    
    # ignore CXNS, CXNR, SCLA
    table = table[~np.in1d(table['type'], ['CXNS', 'CXNR', 'SCLA'])].reset_index(drop=True)
    
    # remove SCG Addition, Release caused by SCG Failure or any other RLFs if needed (default: True)
    if radical:
        table = table[~((table['others'].str.contains('Caused by scg-failure') | table['others'].str.contains('Near after RLF')))].reset_index(drop=True)
    
    # select columns
    selected_cols = ['PCI', 'Cell Identity', 'eNB_ID', 'next_eNB_ID', 'Band ID', 'next_BID', 'DL frequency', 'UL frequency', 'DL bandwidth', 'UL bandwidth', 'TAC', 'MCC', 'MNC']
    table = table[['type', 'start', 'end', 'others', 'm_src', 'm_tgt', 's_src', 's_tgt', 'category', 'inter-eNB', 'inter-Freq', 'band_cng', 'inter-gNB', 'inter-RAT', '4G_5G', 'cause', 'near_before_RLF', *selected_cols]]
    
    return table, new_D

# Input Dataset

In [168]:
root = '/Users/jackbedford/Desktop/MOXA/Code/data'
dates = ['2023-09-12', '2023-09-21', '2023-10-05', '2023-10-19', '2023-10-26']
routes = ['BR', 'BR', 'G', 'R', 'A']
expr_name = 'Bandlock_UDP_9_Phone'
devices = ['sm00', 'sm01']
traces = ['#01', '#02']

rrc_files = {route: [] for route in routes}
dl_files = {route: [] for route in routes}
ul_files = {route: [] for route in routes}
nr_files = {route: [] for route in routes}
lte_files = {route: [] for route in routes}

for date, route in zip(dates, routes):
    for dev in devices:
        for tr in traces:
            data_dir = os.path.join(root, date, expr_name, dev, tr, 'data')
            rrc_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('_rrc.csv')]
            dl_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('udp_dnlk_loss_latency.csv')]
            ul_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('udp_uplk_loss_latency.csv')]
            nr_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('_nr_ml1.csv')]
            lte_files[route] += [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.endswith('ml1.csv') and not s.endswith('nr_ml1.csv')]

In [170]:
dl_files

{'BR': ['/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm00/#01/data/udp_dnlk_loss_latency.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm00/#02/data/udp_dnlk_loss_latency.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm01/#01/data/udp_dnlk_loss_latency.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm01/#02/data/udp_dnlk_loss_latency.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm00/#01/data/udp_dnlk_loss_latency.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm00/#02/data/udp_dnlk_loss_latency.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm01/#01/data/udp_dnlk_loss_latency.csv',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm01/#02/data/udp_dnlk_loss_latency.csv'],
 'G': ['/Users/jackbedford/Desktop/MOXA/C

In [171]:
import pickle

# Specify the file path where you want to save the pickle file
file_path = "./temp/2023-10-31_dl_expr_duras.pkl"

if os.path.isfile(file_path):
    # Load the dictionary from the pickle file
    with open(file_path, 'rb') as f:
        expr_duras = pickle.load(f)
else:
    expr_duras = {route: [] for route in routes}
    
    for key in dl_files.keys():
        for dl_file in dl_files[key]:
            df_dl = set_data(pd.read_csv(dl_file))
            start_time = df_dl.iloc[0]['arr_time']
            end_time = df_dl.iloc[-1]['arr_time']
            dura = (end_time - start_time).total_seconds()
            expr_duras[key].append((start_time, end_time, dura))

    # Save the dictionary to a pickle file
    with open(file_path, 'wb') as f:
        pickle.dump(expr_duras, f)

# pprint(expr_duras)

# Stage 1: 請統計並比較各種event發生的頻率。不同的路線上的發生比率是否類似?

In [188]:
table_ho = pd.DataFrame(columns=['Route', 'HO', 'HO_freq', 'LTEH', 'ENBH', 'MCGH', 'MNBH', 'SCGM', 'Intra-RAT', 'SCGA', 'SCGR-I', 'SCGC-I', 'SCGR-II', 'SCGC-II', 'Inter-RAT', 'Dura'])
table_rlf = pd.DataFrame(columns=['Route', 'RLF', 'RLF_freq', 'MCGF', 'NASR', 'SCGF', 'Dura'])
table_mcgf = pd.DataFrame(columns=['Route', 'MCGF', 'MCGF_freq', 'reconfigurationFailure (0)', 'handoverFailure (1)', 'otherFailure (2)', 'Dura'])
table_nasr = pd.DataFrame(columns=['Route', 'NASR', 'NASR_freq', 'reconfigurationFailure (0)', 'handoverFailure (1)', 'otherFailure (2)', 'Dura'])
table_scgf = pd.DataFrame(columns=['Route', 'SCGF', 'SCGF_freq', 't310-Expiry (0)', 'randomAccessProblem (1)', 'rlc-MaxNumRetx (2)', 'synchReconfigFailureSCG (3)', 'scg-ReconfigFailure (4)', 'srb3-IntegrityFailure (5)', 'other-r16 (6)', 'Dura'])

In [189]:
for key in rrc_files.keys():
    for i, (rrc_file, nr_file, dura) in enumerate(zip(rrc_files[key], nr_files[key], expr_duras[key])):
        print((i, rrc_file, nr_file))
        
        df_nr = pd.read_csv(nr_file)
        if df_nr.empty:
            print('Never detect 5G-NSA signal!!!')
            display(df_nr)
            continue
        
        df_ho, D = mi_parse_handover(pd.read_csv(rrc_file))
        
        start_time = dura[0]
        try:
            end_time = min(dura[1], df_ho[df_ho['cause'] == 'rlc-MaxNumRetx (2)'].iloc[0]['start'])
        except:
            print('Great!! No rlc-MaxNumRetx (2) occurs!!')
            end_time = dura[1]
            
        _dura = (end_time - start_time).total_seconds()
        df_ho = df_ho[(df_ho['start'] >= start_time) & (df_ho['end'] < end_time)].reset_index(drop=True)
        
        new_row_ho = pd.DataFrame([[
            key,
            sum(df_ho['category'] == 'HO'),
            round(sum(df_ho['category'] == 'HO') / dura[2] * 60 * 60, 2),
            sum(df_ho['type'] == 'LTEH'),
            sum(df_ho['type'] == 'ENBH'),
            sum(df_ho['type'] == 'MCGH'),
            sum(df_ho['type'] == 'MNBH'),
            sum(df_ho['type'] == 'SCGM'),
            sum((df_ho['category'] == 'HO') & ~df_ho['inter-RAT']),
            sum(df_ho['type'] == 'SCGA'),
            sum(df_ho['type'] == 'SCGR-I'),
            sum(df_ho['type'] == 'SCGC-I'),
            sum(df_ho['type'] == 'SCGR-II'),
            sum(df_ho['type'] == 'SCGC-II'),
            sum((df_ho['category'] == 'HO') & df_ho['inter-RAT']),
            round(_dura, 3)
        ]], columns=table_ho.columns)

        # display(new_row_ho)
        table_ho = table_ho.append(new_row_ho, ignore_index=True)

        new_row_rlf = pd.DataFrame([[
            key,
            sum(df_ho['category'] == 'RLF'),
            round(sum(df_ho['category'] == 'RLF') / dura[2] * 60 * 60, 2),
            sum(df_ho['type'] == 'MCGF'),
            sum(df_ho['type'] == 'NASR'),
            sum(df_ho['type'] == 'SCGF'),
            round(_dura, 3)
        ]], columns=table_rlf.columns)

        # display(new_row_rlf)
        table_rlf = table_rlf.append(new_row_rlf, ignore_index=True)

        new_row_mcgf = pd.DataFrame([[
            key,
            sum(df_ho['type'] == 'MCGF'),
            round(sum(df_ho['type'] == 'MCGF') / dura[2] * 60 * 60, 2),
            sum((df_ho['type'] == 'MCGF') & (df_ho['cause'] == 'reconfigurationFailure (0)')),
            sum((df_ho['type'] == 'MCGF') & (df_ho['cause'] == 'handoverFailure (1)')),
            sum((df_ho['type'] == 'MCGF') & (df_ho['cause'] == 'otherFailure (2)')),
            round(_dura, 3)
        ]], columns=table_mcgf.columns)

        # display(new_row_mcgf)
        table_mcgf = table_mcgf.append(new_row_mcgf, ignore_index=True)

        new_row_nasr = pd.DataFrame([[
            key,
            sum(df_ho['type'] == 'NASR'),
            round(sum(df_ho['type'] == 'NASR') / dura[2] * 60 * 60, 2),
            sum((df_ho['type'] == 'NASR') & (df_ho['cause'] == 'reconfigurationFailure (0)')),
            sum((df_ho['type'] == 'NASR') & (df_ho['cause'] == 'handoverFailure (1)')),
            sum((df_ho['type'] == 'NASR') & (df_ho['cause'] == 'otherFailure (2)')),
            round(_dura, 3)
        ]], columns=table_nasr.columns)

        # display(new_row_nasr)
        table_nasr = table_nasr.append(new_row_nasr, ignore_index=True)

        new_row_scgf = pd.DataFrame([[
            key,
            sum(df_ho['type'] == 'SCGF'),
            round(sum(df_ho['type'] == 'SCGF') / dura[2] * 60 * 60, 2),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 't310-Expiry (0)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'randomAccessProblem (1)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'rlc-MaxNumRetx (2)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'synchReconfigFailureSCG (3)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'scg-ReconfigFailure (4)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'srb3-IntegrityFailure (5)')),
            sum((df_ho['type'] == 'SCGF') & (df_ho['cause'] == 'other-r16 (6)')),
            round(_dura, 3)
        ]], columns=table_scgf.columns)

        # display(new_row_scgf)
        table_scgf = table_scgf.append(new_row_scgf, ignore_index=True)
        
    #     break
    # break

file_paths = [
    "./temp/2023-10-31_frequency_table_ho.pkl",
    "./temp/2023-10-31_frequency_table_rlf.pkl",
    "./temp/2023-10-31_frequency_table_mcgf.pkl",
    "./temp/2023-10-31_frequency_table_nasr.pkl",
    "./temp/2023-10-31_frequency_table_scgf.pkl"
]

for file_path, table in zip(file_paths, [table_ho, table_rlf, table_mcgf, table_nasr, table_scgf]):
    # Save the dictionary to a pickle file
    with open(file_path, 'wb') as f:
        # print(file_path)
        # display(table)
        pickle.dump(table, f)

(0, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm00/#01/data/diag_log_sm00_2023-09-12_13-34-15_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm00/#01/data/diag_log_sm00_2023-09-12_13-34-15_nr_ml1.csv')


/var/folders/vy/w6g3hxgj76s0gzty27t2d6gm0000gn/T/ipykernel_24928/1761532841.py:11: DtypeWarning: Columns (26,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ho, D = mi_parse_handover(pd.read_csv(rrc_file))


{3050: 7.0, 1750: 3.0}
(1, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm00/#02/data/diag_log_sm00_2023-09-12_14-52-28_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm00/#02/data/diag_log_sm00_2023-09-12_14-52-28_nr_ml1.csv')


/var/folders/vy/w6g3hxgj76s0gzty27t2d6gm0000gn/T/ipykernel_24928/1761532841.py:11: DtypeWarning: Columns (26,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ho, D = mi_parse_handover(pd.read_csv(rrc_file))


{3050: 7.0, 1750: 3.0, 1400: 3.0}
(2, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm01/#01/data/diag_log_sm01_2023-09-12_13-34-15_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm01/#01/data/diag_log_sm01_2023-09-12_13-34-15_nr_ml1.csv')
{3050: 7.0, 1750: 3.0, 3650: 8.0}
(3, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm01/#02/data/diag_log_sm01_2023-09-12_14-52-27_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12/Bandlock_UDP_9_Phone/sm01/#02/data/diag_log_sm01_2023-09-12_14-52-27_nr_ml1.csv')
{3050: 7.0, 1750: 3.0}
(4, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm00/#01/data/diag_log_sm00_2023-09-21_14-29-12_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm00/#01/data/diag_log_sm00_2023-09-21_14-29-12_nr_ml1.csv')


...

(6, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm01/#01/data/diag_log_sm01_2023-09-21_14-29-12_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm01/#01/data/diag_log_sm01_2023-09-21_14-29-12_nr_ml1.csv')


/var/folders/vy/w6g3hxgj76s0gzty27t2d6gm0000gn/T/ipykernel_24928/1761532841.py:11: DtypeWarning: Columns (16,26,40,41,42,43,46,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ho, D = mi_parse_handover(pd.read_csv(rrc_file))


{3050: 7.0, 1750: 3.0, 1400: 3.0}
Great!! No rlc-MaxNumRetx (2) occurs!!
(7, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm01/#02/data/diag_log_sm01_2023-09-21_15-28-46_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-21/Bandlock_UDP_9_Phone/sm01/#02/data/diag_log_sm01_2023-09-21_15-28-46_nr_ml1.csv')
Never detect 5G-NSA signal!!!


,Timestamp,type_id,Raster ARFCN,Num Cells,Serving Cell Index,Serving Cell PCI


(0, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-10-05/Bandlock_UDP_9_Phone/sm00/#01/data/diag_log_sm00_2023-10-05_14-24-16_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-10-05/Bandlock_UDP_9_Phone/sm00/#01/data/diag_log_sm00_2023-10-05_14-24-16_nr_ml1.csv')
{3050: 7.0}
Great!! No rlc-MaxNumRetx (2) occurs!!
(1, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-10-05/Bandlock_UDP_9_Phone/sm00/#02/data/diag_log_sm00_2023-10-05_15-15-00_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-10-05/Bandlock_UDP_9_Phone/sm00/#02/data/diag_log_sm00_2023-10-05_15-15-00_nr_ml1.csv')
{3050: 7.0, 1400: 3.0, 3650: 8.0}
Great!! No rlc-MaxNumRetx (2) occurs!!
(2, '/Users/jackbedford/Desktop/MOXA/Code/data/2023-10-05/Bandlock_UDP_9_Phone/sm01/#01/data/diag_log_sm01_2023-10-05_14-24-16_rrc.csv', '/Users/jackbedford/Desktop/MOXA/Code/data/2023-10-05/Bandlock_UDP_9_Phone/sm01/#01/data/diag_log_sm01_2023-10-05_14-24-16_nr_ml1.csv')
{3050: 7.0}
Great!! No rlc-MaxNumRetx (2) occurs!!
(3, '/

In [190]:
tmp_tables = []
file_paths = [
    "./temp/2023-10-31_frequency_table_ho.pkl",
    "./temp/2023-10-31_frequency_table_rlf.pkl",
    "./temp/2023-10-31_frequency_table_mcgf.pkl",
    "./temp/2023-10-31_frequency_table_nasr.pkl",
    "./temp/2023-10-31_frequency_table_scgf.pkl"
]

for file_path in file_paths:
    # Load the dictionary from the pickle file
    with open(file_path, 'rb') as f:
        tmp_table = pickle.load(f)
        # print(file_path)
        # display(tmp_table)
        tmp_tables.append(tmp_table)

table_ho = tmp_tables[0]
table_rlf = tmp_tables[1]
table_mcgf = tmp_tables[2]
table_nasr = tmp_tables[3]
table_scgf = tmp_tables[4]

In [191]:
display(table_ho)
display(table_rlf)
display(table_mcgf)
display(table_nasr)
display(table_scgf)

,Route,HO,HO_freq,LTEH,ENBH,MCGH,MNBH,SCGM,Intra-RAT,SCGA,SCGR-I,SCGC-I,SCGR-II,SCGC-II,Inter-RAT,Dura
0,BR,260,330.15,44,114,3,12,13,186,71,0,0,1,2,74,2677.356
1,BR,115,142.85,9,39,6,14,22,90,22,0,0,1,2,25,1074.857
2,BR,182,231.10,2,0,25,57,84,168,5,2,1,3,3,14,1510.895
3,BR,105,130.46,0,0,14,38,50,102,1,0,0,1,1,3,855.697
4,BR,194,243.18,45,149,0,0,0,194,0,0,0,0,0,0,2871.952
5,BR,193,241.93,43,150,0,0,0,193,0,0,0,0,0,0,2871.857
6,G,76,126.95,0,0,18,19,39,76,0,0,0,0,0,0,2155.243
7,G,78,132.61,0,0,18,19,41,78,0,0,0,0,0,0,2117.508
8,G,80,133.63,0,0,18,19,43,80,0,0,0,0,0,0,2155.242
9,G,78,132.61,0,0,18,20,40,78,0,0,0,0,0,0,2117.408


,Route,RLF,RLF_freq,MCGF,NASR,SCGF,Dura
0,BR,467,593.00,7,0,460,2677.356
1,BR,222,275.76,7,0,215,1074.857
2,BR,8,10.16,6,0,2,1510.895
3,BR,7,8.70,6,0,1,855.697
4,BR,6,7.52,6,0,0,2871.952
5,BR,7,8.77,7,0,0,2871.857
6,G,0,0.00,0,0,0,2155.243
7,G,2,3.40,1,0,1,2117.508
8,G,0,0.00,0,0,0,2155.242
9,G,1,1.70,0,0,1,2117.408


,Route,MCGF,MCGF_freq,reconfigurationFailure (0),handoverFailure (1),otherFailure (2),Dura
0,BR,7,8.89,0,1,6,2677.356
1,BR,7,8.70,0,0,7,1074.857
2,BR,6,7.62,0,0,6,1510.895
3,BR,6,7.45,0,1,5,855.697
4,BR,6,7.52,0,0,6,2871.952
5,BR,7,8.77,0,0,7,2871.857
6,G,0,0.00,0,0,0,2155.243
7,G,1,1.70,0,0,1,2117.508
8,G,0,0.00,0,0,0,2155.242
9,G,0,0.00,0,0,0,2117.408


,Route,NASR,NASR_freq,reconfigurationFailure (0),handoverFailure (1),otherFailure (2),Dura
0,BR,0,0.00,0,0,0,2677.356
1,BR,0,0.00,0,0,0,1074.857
2,BR,0,0.00,0,0,0,1510.895
3,BR,0,0.00,0,0,0,855.697
4,BR,0,0.00,0,0,0,2871.952
5,BR,0,0.00,0,0,0,2871.857
6,G,0,0.00,0,0,0,2155.243
7,G,0,0.00,0,0,0,2117.508
8,G,0,0.00,0,0,0,2155.242
9,G,0,0.00,0,0,0,2117.408


,Route,SCGF,SCGF_freq,t310-Expiry (0),randomAccessProblem (1),rlc-MaxNumRetx (2),synchReconfigFailureSCG (3),scg-ReconfigFailure (4),srb3-IntegrityFailure (5),other-r16 (6),Dura
0,BR,460,584.11,1,459,0,0,0,0,0,2677.356
1,BR,215,267.07,0,214,0,1,0,0,0,1074.857
2,BR,2,2.54,0,1,0,1,0,0,0,1510.895
3,BR,1,1.24,0,1,0,0,0,0,0,855.697
4,BR,0,0.00,0,0,0,0,0,0,0,2871.952
5,BR,0,0.00,0,0,0,0,0,0,0,2871.857
6,G,0,0.00,0,0,0,0,0,0,0,2155.243
7,G,1,1.70,0,1,0,0,0,0,0,2117.508
8,G,0,0.00,0,0,0,0,0,0,0,2155.242
9,G,1,1.70,0,0,0,1,0,0,0,2117.408


In [192]:
root = '/Users/jackbedford/Desktop/MOXA/Code/results/2023-10-31 results'
file_paths = [
    "./2023-10-31_frequency_table_ho.csv",
    "./2023-10-31_frequency_table_rlf.csv",
    "./2023-10-31_frequency_table_mcgf.csv",
    "./2023-10-31_frequency_table_nasr.csv",
    "./2023-10-31_frequency_table_scgf.csv"
]

for file_path, table in zip(file_paths, [table_ho, table_rlf, table_mcgf, table_nasr, table_scgf]):
    # Save the dictionary to a pickle file
    table.to_csv(os.path.join(root, file_path), index=False)